# Create monthly CESM atmospheric deposition files

NCAR CESM output:
Community Earth System Model (Community Atmosphere Model - CAM https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1002/2013MS000279) output: https://www.earthsystemgrid.org/; CESM1 CAM5 BGC Large Ensemble Atmosphere Post Processed Data, Monthly Averages.

Specific run output: https://www.earthsystemgrid.org/dataset/ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.atm.proc.monthly_ave.html

In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import matplotlib
import modules.mapping_functions as mf
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
print('Run id: ucar.cgd.ccsm4.CESM_CAM5_BGC_LE')

Run id: ucar.cgd.ccsm4.CESM_CAM5_BGC_LE


For Mode MAM3 (https://www.geosci-model-dev.net/5/709/2012/):

- a1 --- Aitken mode --- 0.015-0.053 μm
- a2 --- Accumulation mode --- 0.058-0.27 μm
- a3 --- Coarse mode --- 0.80-3.65 μm

#### Load files

In [3]:
mesh  = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask = np.array(mesh.variables['tmask'])
tmask = np.array(tmask[0,:,:,:])
mlons = np.array(mesh.variables['nav_lon'])
mlats = np.array(mesh.variables['nav_lat'])
Z_masked = np.ma.masked_where((tmask > 0.1), tmask) 

#### Functions

In [4]:
def save_file(filename, field1, field2, field3):
    ncd = nc.Dataset(filename, 'w', zlib=True)
    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # variables
    fine_dust             = ncd.createVariable('fdust', 'float64', ('y','x'))
    fine_dust.units       = 'kg/m2/s'
    fine_dust.long_name   = 'Fine dust deposition flux'  
    fine_dust.coordinates = 'nav_lon nav_lat'
    fine_dust[:]          = field1
    
    coarse_dust             = ncd.createVariable('cdust', 'float64', ('y','x'))
    coarse_dust.units       = 'kg/m2/s'
    coarse_dust.long_name   = 'Coarse dust deposition flux'  
    coarse_dust.coordinates = 'nav_lon nav_lat'
    coarse_dust[:]          = field2
    
    black_c             = ncd.createVariable('bc', 'float64', ('y','x'))
    black_c.units       = 'kg/m2/s'
    black_c.long_name   = 'Black carbon deposition flux'  
    black_c.coordinates = 'nav_lon nav_lat'
    black_c[:]          = field3
    
    print('saved ', filename)

    ncd.close()
    return

In [7]:
def load_data():
    folder = '/ocean/brogalla/GEOTRACES/data/NCAR/'

    dst_a1D = nc.Dataset(f'{folder}merged_dst_a1DDF.nc')
    dst_a1S = nc.Dataset(f'{folder}merged_dst_a1SFWET.nc')
    dst_a3D = nc.Dataset(f'{folder}merged_dst_a3DDF.nc')
    dst_a3S = nc.Dataset(f'{folder}merged_dst_a3SFWET.nc')
    bc_a1D  = nc.Dataset(f'{folder}merged_bc_a1DDF.nc')
    bc_a1S  = nc.Dataset(f'{folder}merged_bc_a1SFWET.nc')
    
    dst_a1DDF   = np.array(dst_a1D.variables['dst_a1DDF'])
    dst_a1SFWET = np.array(dst_a1S.variables['dst_a1SFWET'])
    dst_a3DDF   = np.array(dst_a3D.variables['dst_a3DDF'])
    dst_a3SFWET = np.array(dst_a3S.variables['dst_a3SFWET'])
    BC_a1DDF    = np.array(bc_a1D.variables['bc_a1DDF'])
    BC_a1SFWET  = np.array(bc_a1S.variables['bc_a1SFWET'])
    
    # zero negative deposition fluxes:
    dst_a1DDF[dst_a1DDF < 0]     = 0
    dst_a1SFWET[dst_a1SFWET < 0] = 0
    dst_a3DDF[dst_a3DDF < 0]     = 0
    dst_a3SFWET[dst_a3SFWET < 0] = 0
    BC_a1DDF[BC_a1DDF < 0]       = 0
    BC_a1SFWET[BC_a1SFWET < 0]   = 0
    
    lon  = dst_a1D.variables['lon']
    lat  = dst_a1D.variables['lat']
    date = dst_a1D.variables['date']
    
    lon = np.array(lon)
    for i in range(0,len(lon)):
        if lon[i] >= 180:
            lon[i] = -360+lon[i]
            
    # change lons and lats array dimensions: (192x288)
    lons, lats = np.meshgrid(lon,lat)
    lons=np.array(lons)
    lats=np.array(lats)    
    
    dst_a1 = np.add(dst_a1DDF[:,:,:], dst_a1SFWET[:,:,:])
    dst_a3 = np.add(dst_a3DDF[:,:,:], dst_a3SFWET[:,:,:])
    bc_a1  = np.add(BC_a1DDF[:,:,:], BC_a1SFWET[:,:,:])
    
    return date, lons, lats, dst_a1, dst_a3, bc_a1

In [8]:
def find_dates(file_year):
    file_date_start = file_year*10000 + 101
    file_date_end = file_year*10000 + 1201
    
    start_index = []
    end_index = []

    for i in range(0,len(date)):
        if date[i] == file_date_start:
            start_index = i
        elif date[i] == file_date_end:
            end_index = i

    # print('start index: ', start_index)
    # print('end index: ', end_index)
    
    return start_index, end_index

In [9]:
def data_to_ANHA12(file_year, savefiles=False):
    
    si, ei = find_dates(file_year)
    dusta1 = dust_a1[si:ei+1,:,:]
    dusta3 = dust_a3[si:ei+1,:,:]
    bca1   = bc_a1[si:ei+1,:,:]
    print('Check that output is of the correct dimensions: ', dusta1.shape, dusta3.shape, bca1.shape)
    
    interp_dst_a1 = np.empty((12, 2400, 1632))
    interp_dst_a3 = np.empty((12, 2400, 1632))
    interp_bc_a1  = np.empty((12, 2400, 1632))

    # loop over the months:
    for i in range(0,12):
        interp_dst_a1[i,:,:] = mf.interp_np(lons, lats, dusta1[i,:,:], mlons, mlats)
        interp_dst_a3[i,:,:] = mf.interp_np(lons, lats, dusta3[i,:,:], mlons, mlats)
        interp_bc_a1[i,:,:]  = mf.interp_np(lons, lats, bca1[i,:,:], mlons, mlats)
        
    if savefiles:
        location = '/ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/'
        
        for i in range(1,13):
            save_file(f'{location}atm_flux_y{file_year}m{i:02}.nc',interp_dst_a1[i-1,:,:],\
                      interp_dst_a3[i-1,:,:], interp_bc_a1[i-1,:,:])
    
    return interp_dst_a1, interp_dst_a3, interp_bc_a1

#### Interpolate to ANHA12 grid:

In [10]:
date, lons, lats, dust_a1, dust_a3, bc_a1 = load_data()

In [14]:
for year in np.arange(2013,2021,1):
    print(year)
    interp_dst_a1, interp_dst_a3, interp_bc_a1 = data_to_ANHA12(year, savefiles=True)

2013
Check that output is of the correct dimensions:  (12, 192, 288) (12, 192, 288) (12, 192, 288)
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m01.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m02.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m03.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m04.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m05.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m06.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m07.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m08.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m09.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/atm_flux_y2013m10.nc
saved  /oce

### Next time, plot out dust forcing fields and check whether any negative or NaN or extreme values